<h1>YOLO</h1>

[Details zur Funktionsweise von YOLO folgen]

<h2>1. Install</h2>

Um nicht alles per Hand zu programmieren, gibt es Libraries, die eine bequeme Nutzung von YOLO erlauben.

Als Erstes muss PyTorch installiert werden. <br>
> https://pytorch.org/get-started/locally/ [Letzter Zugriff: 08.07.2024]

Folge diesen Anweisungen, um Ultralytics zu installieren: <br>
> https://github.com/ultralytics/ultralytics [Letzter Zugriff: 08.07.2024] <br>
> https://docs.ultralytics.com/quickstart/   [Letzter Zugriff: 08.07.2024]


Danach kann dieser Befehl in die Konsole eingegeben werden: <br>
`yolo predict model=yolov8s.pt source='https://ultralytics.com/images/bus.jpg'` <br>

Damit wird ein Bild heruntergeladen und direkt für die Predicion genutzt. Dabei wird ein Pre-Trained Model verwendet.

<u>Hinweis</u>:<br>
Um diese Prediction auszuführen, müssen alle Libraries verfügbar sein. <br>
Nicht vorhandene Libraries können z. B. mit `pip install <package>==<version>` installiert werden. <br>
- Bei Numpy kann der folgende Fehler auftreten: `ImportError: numpy.core.multiarray failed to import`. Um das zu beheben muss eine spezifische Numpy Version installiert werden (hier: 1.26.0 statt 2.0.0).
- Dasselbe gilt auf für andere Packages.

In [1]:
# Befehl kann auch direkt im Notebook ausgeführt werden.
!yolo predict model=yolov8s.pt source='https://ultralytics.com/images/bus.jpg'

Ultralytics YOLOv8.2.51 ðŸš€ Python-3.12.4 torch-2.3.1+cpu CPU (Intel Core(TM) i9-9900K 3.60GHz)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 C:\Users\Alexander\Desktop\ML_DL_self_learn\GitHub\ML_DL_Content\deep_learning\PyTorch\projects\1__YOLO\bus.jpg: 640x480 4 persons, 1 bus, 378.8ms
Speed: 5.0ms preprocess, 378.8ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict2
ðŸ’¡ Learn more at https://docs.ultralytics.com/modes/predict


Das Ergebnis: <br>

Linkes Bild: Iput <br>
Rechtes Bild: Output<br>

<table>
    <tr>
      <td>
      <img src="bus.jpg" hight=300 width=300>
      </td>
      <td>
        <img src="./runs/detect/predict/bus.jpg"  hight=300 width=300>
      </td>
     </tr>
</table>

Wie geht das?: <br>
Das Netzwerk gibt ein Vektor aus, wo unter anderem die Daten für die Boundaries stehen. Diese Müssen nach Klassen in das Bild eingezeichnet werden. <br>
Vor dem Einzeichnen wird ein Algorithmus names <u>IOU</u> verwendet, um überlappenden Boxen zu einer zu verschmelzen.
- Zudem werden noch andere Daten ausgegeben, wie: 4 persons, 1 bus, ..., siehe unten.

Das alles passiert intern, und als Ausgabe bekommen wir das Bild. Die eigentliche Ausgabe des Netzes ist ein Vektor, bestehend aus mehreren sogenannten <u>Anchor Boxen<u>. <br>

Siehe auch: <br>
> YOLOv3 From Scratch Using PyTorch: <br>
> https://www.geeksforgeeks.org/yolov3-from-scratch-using-pytorch/ [Letzter Zugriff: 08.07.2024]


In [2]:
## Als Code: ## 
from ultralytics import YOLO

# Lade Model Typ Nano. 
# - 8n: YOLO Nano, es gibt noch s: small, L: Large und x: extra large.
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Führe Prediction aus. 
results = model("bus.jpg")  


image 1/1 C:\Users\Alexander\Desktop\ML_DL_self_learn\GitHub\ML_DL_Content\deep_learning\PyTorch\projects\1__YOLO\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 153.0ms
Speed: 3.0ms preprocess, 153.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
